# Video Processing for BT Lab
Functions to check if run is good
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import json
import datetime
import numpy as np
import pandas as pd
import traceback
import cv2

In [2]:
# make sure to update path
user_drive = input("Enter user drive: ").upper()
video_path = f"{user_drive}:/DI_centre_structured"
input(f"Is this the right directory - {video_path}?")

''

In [3]:
VIDEO_CHARACTERISTICS = {
    "With Blankets" : "WB",
    "B" : "WB",
    "Without Blankets" : "WOB",
    "WOB": "WOB",
    "3 Meters" : "3m",
    "2 Meters" : "2m",
    "Hold Breath" : "HB",
    "Hold Breathe" : "HB",
    "H" : "HB",
    "Relaxed" : "rel",
    "R": "rel",
}

In [4]:
#local dirs
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"
log_dir = repo_dir + "/records/logs"

In [5]:
""" gets folder paths """
def get_frames_path(local_path: str, level:str) -> list["str"]:
    fixed_path = local_path.replace("\\", "/")
    fixed_path_split = fixed_path.split("/")
    video_folder = "/".join(fixed_path_split[:-1])
    video_filename = fixed_path_split[-1].split(".")[0]
    folder_path = video_folder + f"/frames_{video_filename}_{level}"
    return [folder_path, video_folder]

In [6]:
""" gets folder paths """
def get_frames_path(local_path: str, level:str) -> list["str"]:
    fixed_path = local_path.replace("\\", "/")
    fixed_path_split = fixed_path.split("/")
    video_folder = "/".join(fixed_path_split[:-1])
    video_filename = fixed_path_split[-1].split(".")[0]
    folder_path = video_folder + f"/frames_{video_filename}_{level}"
    return [folder_path, video_folder]

In [7]:
""" reads JSON file """
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

In [8]:
""" re samples number of frames """
def up_sample(old_fps: int, new_fps: int, start:int) -> list[int]:
    frames_arr = np.arange(start, old_fps, dtype=int)
    frames_interp = np.linspace(start, old_fps-1, new_fps)
    nearest_indices = np.round(frames_interp).astype(int)
    up_sampled_list = np.take(frames_arr, nearest_indices, mode='wrap')

    return up_sampled_list.tolist()

In [9]:
def get_id(patient_data: dict, video_count:int) -> str:
    alias = patient_data["alias"]
    blanket = VIDEO_CHARACTERISTICS.get(patient_data["blanket"], "?")
    distance = VIDEO_CHARACTERISTICS.get(patient_data["distance"].title(), "?")
    breathing = VIDEO_CHARACTERISTICS.get(patient_data["breathing"], "?")
    id = alias + "-" + distance + "-" + blanket + "-" + breathing
    id = f"{alias}_{video_count}-{distance}-{blanket}-{breathing}"
    return id

In [18]:
def run_folder_check(video_path:str , save_folder:str, frame_frequency: dict, new_fps:int, patient_id:str) -> list["str"]:
    # open video file
    video = cv2.VideoCapture(video_path)
    assert video.isOpened()

    # unpack metadata
    vid_fps = int(video.get(cv2.CAP_PROP_FPS))
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    vid_length_floored = number_of_frames // vid_fps

    # local variables
    expected_frames = int(new_fps*vid_length_floored)
    number_of_files_in_folder = int(len(os.listdir(save_folder)))

    if not os.path.exists(save_folder):
        print(f"{save_folder} does not exists.")
        return save_folder

    if number_of_files_in_folder != expected_frames:
        print(f"{save_folder} - Not enough frames in folder. Only found {number_of_files_in_folder} files. Expecting {expected_frames}.")
        return save_folder

    video.release()

    return None
    

In [11]:
def check_if_empty_folders(all_patients:dict, level: str, new_fps:int, user_drive:str) -> list[str]:
    visited_folders = {}    
    rerun_folders = []

    for json_index, patient_info in all_patients.items():
        try:
            video_path = patient_info["local path"]
            old_fps = int(patient_info["old fps"])

            frames_folder, video_folder = get_frames_path(video_path, level)
            visited_folders[video_folder] = visited_folders.get(video_folder, 0) + 1
            video_count = visited_folders[video_folder]

            frames_to_pick = up_sample(old_fps, new_fps, 1)
            frames_idx = pd.Index(frames_to_pick, name="frames")
            frame_frequency = frames_idx.value_counts()

            if len(frames_to_pick) != new_fps:
                raise ValueError("Number of frames to pick is not equal to new fps")

            # get id
            patient_id = get_id(patient_info, video_count)

            # run video to frames
            rerun_folder = run_folder_check(video_path, frames_folder, frame_frequency, new_fps, patient_id)

            if rerun_folder != None:
                rerun_folders.append(rerun_folder)

        except Exception as e:
            traceback.print_exc()
            print(f'''{type(e)}: {e} for video {patient_info["filename"]}''')

    return rerun_folders

# Run tests

In [12]:
""" local vars"""

rgb_fps = {
    "lower_bound": 10,
    "upper_bound": 20
}

thermal_fps = {
    "lower_bound": 5,
    "upper_bound": 10
}

In [13]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "/rgb_complete.json")
metadata_thermal = load_json(json_dir, "/thermal_complete.json")

In [14]:
""" check videos (rgb) """
all_rgb_folders = []

for level, new_fps in rgb_fps.items():
    print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
    rerun_folders = check_if_empty_folders(metadata_rgb, level, new_fps, user_drive)
    all_rgb_folders.append(rerun_folders)


Adjusting FPS to 10

Adjusting FPS to 20
D:/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/frames_Arun2_upper_bound - Not enough frames in folder. Only found 119 files.


In [15]:
all_rgb_folders

[[],
 ['D:/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/frames_Arun2_upper_bound']]

In [19]:
""" check videos (thermal) """
all_thermal_folders = []

for level, new_fps in rgb_fps.items():
    print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
    rerun_folders = check_if_empty_folders(metadata_thermal, level, new_fps, user_drive)
    all_thermal_folders.append(rerun_folders)


Adjusting FPS to 10
D:/DI_centre_structured/DI_THERMAL_FLIR//1/2 meters/H/WOB/frames_1_H_WOB_2m_lower_bound - Not enough frames in folder. Only found 1075 files. Expecting 2150.
D:/DI_centre_structured/DI_THERMAL_FLIR//1/2 meters/H/WOB/frames_1_H_WOB_2m_2_lower_bound - Not enough frames in folder. Only found 410 files. Expecting 820.
D:/DI_centre_structured/DI_THERMAL_FLIR//1/2 meters/R/WOB/frames_1_R_WOB_2m_lower_bound - Not enough frames in folder. Only found 310 files. Expecting 620.
D:/DI_centre_structured/DI_THERMAL_FLIR//10/2 meters/H/B/frames_10_holdingbreath_blanket_2me-12222022145955_lower_bound - Not enough frames in folder. Only found 905 files. Expecting 1810.
D:/DI_centre_structured/DI_THERMAL_FLIR//10/2 meters/H/WOB/frames_10_H_WOB_2m_lower_bound - Not enough frames in folder. Only found 990 files. Expecting 1980.
D:/DI_centre_structured/DI_THERMAL_FLIR//10/2 meters/R/B/frames_10_relaxed_blanket_2me-12222022145805_lower_bound - Not enough frames in folder. Only found 925

Traceback (most recent call last):
  File "C:\Users\BT-Lab-Helder\AppData\Local\Temp\ipykernel_15032\3987211603.py", line 25, in check_if_empty_folders
    rerun_folder = run_folder_check(video_path, frames_folder, frame_frequency, new_fps, patient_id)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\BT-Lab-Helder\AppData\Local\Temp\ipykernel_15032\4070635065.py", line 13, in run_folder_check
    number_of_files_in_folder = int(len(os.listdir(save_folder)))
                                        ^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/DI_centre_structured/DI_THERMAL_FLIR//15/2 meters/H/B/frames_output_video_lower_bound'


D:/DI_centre_structured/DI_THERMAL_FLIR//15/2 meters/R/WOB/frames_15_relaxed_wob_2meters-12232022085332_lower_bound - Not enough frames in folder. Only found 20 files. Expecting 50.
D:/DI_centre_structured/DI_THERMAL_FLIR//15/2 meters/R/WOB/frames_15_relaxed_wob_2meters-12232022085344_lower_bound - Not enough frames in folder. Only found 60 files. Expecting 150.
D:/DI_centre_structured/DI_THERMAL_FLIR//15/2 meters/R/WOB/frames_15_relaxed_wob_2meters-12232022085400_lower_bound - Not enough frames in folder. Only found 72 files. Expecting 180.
D:/DI_centre_structured/DI_THERMAL_FLIR//2/2 meters/H/B/frames_2_H_B_2m-12222022100256_lower_bound - Not enough frames in folder. Only found 760 files. Expecting 1520.
D:/DI_centre_structured/DI_THERMAL_FLIR//2/2 meters/H/WOB/frames_2_H_WOB_2m-12222022095411_lower_bound - Not enough frames in folder. Only found 740 files. Expecting 1480.
D:/DI_centre_structured/DI_THERMAL_FLIR//2/2 meters/R/B/frames_2_R_B_2m-12222022095917_lower_bound - Not enough 

Traceback (most recent call last):
  File "C:\Users\BT-Lab-Helder\AppData\Local\Temp\ipykernel_15032\3987211603.py", line 25, in check_if_empty_folders
    rerun_folder = run_folder_check(video_path, frames_folder, frame_frequency, new_fps, patient_id)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\BT-Lab-Helder\AppData\Local\Temp\ipykernel_15032\4070635065.py", line 13, in run_folder_check
    number_of_files_in_folder = int(len(os.listdir(save_folder)))
                                        ^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/DI_centre_structured/DI_THERMAL_FLIR//15/2 meters/H/B/frames_output_video_upper_bound'


D:/DI_centre_structured/DI_THERMAL_FLIR//2/2 meters/H/WOB/frames_2_H_WOB_2m-12222022095411_upper_bound - Not enough frames in folder. Only found 1480 files. Expecting 2960.
D:/DI_centre_structured/DI_THERMAL_FLIR//2/2 meters/R/B/frames_2_R_B_2m-12222022095917_upper_bound - Not enough frames in folder. Only found 1560 files. Expecting 3120.
D:/DI_centre_structured/DI_THERMAL_FLIR//2/2 meters/R/WOB/frames_2_R_WOB_2m-12222022095034_upper_bound - Not enough frames in folder. Only found 1440 files. Expecting 2880.
D:/DI_centre_structured/DI_THERMAL_FLIR//2/3 meters/H/B/frames_2_H_B_3m-12222022102602_upper_bound - Not enough frames in folder. Only found 1330 files. Expecting 2660.
D:/DI_centre_structured/DI_THERMAL_FLIR//2/3 meters/H/WOB/frames_2_H_WOB_3m-12222022101944_upper_bound - Not enough frames in folder. Only found 1310 files. Expecting 2620.
D:/DI_centre_structured/DI_THERMAL_FLIR//2/3 meters/R/B/frames_2_R_B_3m-12222022102314_upper_bound - Not enough frames in folder. Only found 12